In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import LocalOutlierFactor
data_path = './open/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path+'test.csv')
# valid = pd.read_csv(data_path+'valid.csv')
valid = pd.read_csv(data_path+'val.csv')
submission =pd.read_csv(data_path+'sample_submission.csv')



In [2]:
sample = train.sample(frac = 0.1)
train = train.drop('ID',axis=1)
columns = sample.columns.tolist()
columns
X = sample[columns]

print(X)



            ID        V1        V2        V3        V4        V5        V6  \
32727    81444 -0.971650  0.790213  1.616566  0.014234  0.545193 -0.517151   
22620    56278  1.336730  0.741224 -0.916212  1.015457  0.832310 -0.425570   
76235   190324 -0.211622  0.855239 -0.531019  0.520083  1.186700 -0.766705   
22210    55265  1.124823 -0.058563  0.609786  0.794951 -0.634184 -0.497233   
28075    69870 -1.490965  0.420934  1.444110  0.461164 -0.166412  0.620151   
...        ...       ...       ...       ...       ...       ...       ...   
36120    89653 -0.403510 -3.203221 -0.076343 -0.838460 -1.683876  0.629251   
87301   218172 -3.248182  3.473824 -3.763077 -1.513779 -0.416635 -1.205835   
111033  277818  2.007403  0.076843 -1.798019  1.188488  0.591914 -0.655670   
79841   199270  1.901739 -0.610596 -0.811448  0.696854 -0.593986 -0.701132   
35712    88661  0.744559 -1.484757 -0.010371 -0.501087 -1.215600 -0.660886   

              V7        V8        V9  ...       V21       V22  

In [3]:
sample = sample.drop('ID',axis=1)

In [4]:
outlier_fraction = 0.001

classifiers= IsolationForest(n_estimators=100, max_samples=len(sample),contamination=outlier_fraction,verbose=0).fit(sample)

/Users/eunsoo/miniforge3/envs/pytorch-dev/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [5]:
y_prediction = classifiers.predict(sample)
y_prediction[y_prediction==1] = 0
y_prediction[y_prediction==-1] = 1


In [6]:
y_prediction
ind = np.where(y_prediction!=1)
# print(ind)
print(y_prediction[ind])

# print(y_prediction.shape)


[0 0 0 ... 0 0 0]


In [7]:
valid.head()
#TODO : 앙상블 만들기,autoencoder parameter search(grid)해서 최적 찾기, ID 행 제외


,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V22,V23,V24,V25,V26,V27,V28,V29,V30,Class
0,10,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,-0.255991,-0.994878,0
1,22,0.962496,0.328461,-0.171479,2.109204,1.129566,1.696038,0.107712,0.521502,-1.191311,...,0.402492,-0.048508,-1.371866,0.390814,0.199964,0.016371,-0.014605,0.168937,-0.994784,0
2,63,1.145524,0.575068,0.194008,2.598192,-0.092210,-1.044430,0.531588,-0.241888,-0.896287,...,-0.119703,-0.076510,0.691320,0.633984,0.048741,-0.053192,0.016251,0.169496,-0.994502,0
3,69,0.927060,-0.323684,0.387585,0.544474,0.246787,1.650358,-0.427576,0.615371,0.226278,...,0.079359,0.096632,-0.992569,0.085096,0.377447,0.036096,-0.005960,0.331307,-0.994467,0
4,83,-3.005237,2.600138,1.483691,-2.418473,0.306326,-0.824575,2.065426,-1.829347,4.009259,...,-0.181268,-0.163747,0.515821,0.136318,0.460054,-0.251259,-1.105751,-0.287012,-0.994373,0


In [8]:
fraud = valid[valid['Class']==1]
normal = valid[valid['Class']==0]
ratio_fraud = len(fraud)/len(normal)
print(ratio_fraud)

0.0010551491277433877


In [9]:
ratio_fraud
X = valid.drop(['Class','ID'],axis=1)
Y = valid['Class']

valid_predict = classifiers.predict(X)


In [10]:
valid_predict[valid_predict==1] = 0
valid_predict[valid_predict==-1] = 1

In [11]:
print(accuracy_score(valid_predict,Y))
print(classification_report(valid_predict,Y))

0.9990513667345935
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28443
           1       0.37      0.58      0.45        19

    accuracy                           1.00     28462
   macro avg       0.68      0.79      0.72     28462
weighted avg       1.00      1.00      1.00     28462



In [12]:
test.head()


,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,AAAA0x1,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,1.783274,-0.994983
1,AAAA0x2,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.269825,-0.994983
2,AAAA0x5,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0.670579,-0.994960
3,AAAA0x7,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,-0.237686,-0.994937
4,AAAA0xc,0.384978,0.616109,-0.874300,-0.094019,2.924584,3.317027,0.470455,0.538247,-0.558895,...,0.049924,0.238422,0.009130,0.996710,-0.767315,-0.492208,0.042472,-0.054337,-0.167819,-0.994866


In [13]:
# test_prediction = classifiers.predict(test)
test = test.drop('ID',axis=1)
test_prediction = classifiers.predict(test)
test_prediction[test_prediction==1] = 0
test_prediction[test_prediction==-1] = 1


In [14]:
np.unique(test_prediction)

array([0, 1])

In [15]:
len(test_prediction[test_prediction==1])

120

In [16]:
# test_prediction
df = pd.DataFrame(test_prediction, columns = ['Class'])




In [17]:
sum_df = pd.concat([submission['ID'],df['Class']],axis=1)

In [18]:
sum_df.head()

,ID,Class
0,AAAA0x1,0
1,AAAA0x2,0
2,AAAA0x5,0
3,AAAA0x7,0
4,AAAA0xc,0


In [19]:
submission.head()
# sum_df.to_csv('faud_card.csv',index=False)


,ID,Class
0,AAAA0x1,1
1,AAAA0x2,1
2,AAAA0x5,1
3,AAAA0x7,1
4,AAAA0xc,1


In [20]:
import torch
import torchvision
from torchvision import transforms
import torch.nn.functional as F

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset 
import numpy as np
import cv2
import matplotlib.pyplot as plt

data_path = './open/'
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path+'test.csv')
# valid = pd.read_csv(data_path+'valid.csv')
valid = pd.read_csv(data_path+'val.csv')
submission =pd.read_csv(data_path+'sample_submission.csv')

In [21]:
device = "mps" if torch.backends.mps.is_available() else "cpu"


In [22]:
class Flatten(torch.nn.Module): # 4D -> 2D로 계산하기
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1) # (배치 수, 채널 수, 이미지 너비, 이미지 높이) -> (배치 수, 채널 수*이미지 너비*이미지 높이)

class Deflatten(nn.Module): # 2D -> 4D로 계산하기

    def __init__(self, k):
        super(Deflatten, self).__init__()
        self.k = k
        
    def forward(self, x):
        s = x.size()
        
        # 벡터 사이즈 = 채널 수*이미지 너비*이미지 높이
        # 벡터 사이즈 = 채널 수*이미지 사이즈**2
        # 이미지 사이즈 = (벡터 사이즈//채널 수)**.5
        feature_size = int((s[1]//self.k)**.5) 
        
        return x.view(s[0],self.k,feature_size,feature_size) # (배치 수, 채널 수*이미지 너비*이미지 높이) -> (배치 수, 채널 수, 이미지 너비, 이미지 높이) 
    

class Autoencoder(nn.Module):
    def __init__(self,input_dim):
        super(Autoencoder, self).__init__()
        self.input_dim = input_dim
        # k = 16
        self.encoder = nn.Sequential(
                        nn.Linear(input_dim, 16, bias=True),
                        nn.ReLU(),
                        nn.Linear(16, 8, bias=True),
                        nn.ReLU(),
                        nn.Linear(8, 4, bias=True),
                        nn.ReLU(),
                        nn.Linear(4, 2, bias=True),
                        nn.ReLU(),
                    
        )
        
        self.decoder = nn.Sequential(
                        nn.Linear(2, 4, bias=True),
                        nn.ReLU(),  
                        nn.Linear(4, 8, bias=True),
                        nn.ReLU(),  
                        nn.Linear(8, 16, bias=True),
                        nn.ReLU(),  
                        nn.Linear(16, input_dim, bias=True),
                        nn.ReLU()
        )
    
    def forward(self, x):
        
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        return decoded

In [23]:
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.pipeline import Pipeline

# configure our pipeline

pipeline = Pipeline([('normalizer', Normalizer()),
                     ('scaler', MinMaxScaler())])

In [24]:
valid.head()
valid_x = valid.drop(['ID','Class'],axis=1)


In [25]:


pipeline.fit(train)
pipeline.fit(valid_x)


Pipeline(steps=[('normalizer', Normalizer()), ('scaler', MinMaxScaler())])

In [26]:
train.head()
train = train.drop('ID',axis=1)

In [27]:
X_train_transformed = pipeline.transform(train)
X_valid_transformed = pipeline.transform(valid_x)
# X_validate_transformed = pipeline.transform(X_validate)

In [28]:
input_dim = X_train_transformed.shape[1]

model = Autoencoder(input_dim).to(device)


In [29]:
class TensorData(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index] 

    def __len__(self):
        return self.len

In [30]:
# print(type(X_train_transformed))

X_train_transformed


array([[0.41701482, 0.41285177, 0.5927302 , ..., 0.40679479, 0.70683233,
        0.32016843],
       [0.39155868, 0.51349321, 0.70320289, ..., 0.43433993, 0.40624485,
        0.20231822],
       [0.42827177, 0.81691873, 0.72107753, ..., 0.4556965 , 0.03288484,
        0.01038565],
       ...,
       [0.47452355, 0.74289961, 0.53813482, ..., 0.45507561, 0.0429323 ,
        0.96978286],
       [0.56015925, 0.78895979, 0.31867664, ..., 0.4473469 , 0.03503791,
        0.94210336],
       [0.18292212, 0.88717994, 0.15381295, ..., 0.46422359, 0.10767664,
        0.54428775]])

In [31]:

from sklearn.model_selection import train_test_split # 전체 데이터를 학습 데이터와 평가 데이터로 나눈다.

# train, valid = train_test_split(train,test_size=0.1,stratify=train[['healthy','multiple_diseases','rust','scab']])
train, valid = train_test_split(X_train_transformed, test_size=0.1)


In [32]:
trainsets = TensorData(train, train)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=256, shuffle=True)



testsets = TensorData(valid, valid)
testloader = torch.utils.data.DataLoader(testsets, batch_size=256, shuffle=False)

In [33]:
criterion = nn.MSELoss() # MSE 사용
optimizer = optim.AdamW(model.parameters(), lr=7e-5)

In [34]:
X_train_transformed.shape


(113842, 30)

In [35]:
Epochs = 100
# for epoch in range(Epochs):

#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):

#         inputs = data[0].to(device)

#         optimizer.zero_grad()
#         outputs = model(inputs)
        
#         loss = criterion(inputs, outputs) # 라벨 대신 입력 이미지와 출력 이미지를 비교

#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()


#     cost = running_loss / len(trainloader)        
    
#     if epoch % 10 == 0:
#         print('[%d] loss: %.3f' %(epoch + 1, cost))  
#         # check_plot()

In [36]:
from transformers import get_cosine_schedule_with_warmup
scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps = len(trainloader)*3,num_training_steps = len(trainloader)*Epochs)


In [39]:
from tqdm.notebook import tqdm
from sklearn.metrics import mean_squared_error # Regression 문제의 평가를 위해 MSE(Mean Squared Error)를 불러온다.
import numpy as np
valid_loss_min = np.inf
for epoch in range(100):
  model.train()
  train_loss = 0
  for input_data,labels in tqdm(trainloader):
    inputs = input_data.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)

    loss = criterion(outputs,labels)
    loss.backward()
    train_loss +=loss.item()
    optimizer.step()
    scheduler.step()
  if epoch %10 ==0:
    print(f'{epoch+1} train loss : {train_loss}')

  model.eval()
  epoch_valid_loss = 0
  preds_list =[]
  true_onehot_list = []
  with torch.no_grad():
      for inputs,labels in testloader:
         
          inputs = inputs.to(device)
          labels = labels.to(device)
          outputs = model(inputs)
          # print(outputs)
          loss = criterion(outputs,labels)
          epoch_valid_loss +=loss.item()
  
  print(f'valid loss : {epoch_valid_loss/len(testloader):.4f}')
  if epoch_valid_loss<=valid_loss_min:
    print("save model..")
    torch.save(model.state_dict(),'./model_state_dict.pth')
    valid_loss_min = epoch_valid_loss
    
    

  0%|          | 0/401 [00:00<?, ?it/s]

1 train loss : 22.60694921761751


NameError: name 'lables' is not defined

In [ ]:
from sklearn.metrics import mean_squared_error # Regression 문제의 평가를 위해 MSE(Mean Squared Error)를 불러온다.
import numpy as np

model.eval()
epoch_valid_loss = 0
preds_list =[]
true_onehot_list = []
with torch.no_grad():
    for data in testloader:
        inputs, values = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        # print(outputs)
        loss = criterion(outputs,inputs)
        epoch_valid_loss +=loss.item()
        # preds_list.extend()
        # print(outputs.cpu().numpy())
        preds = outputs.cpu().numpy()
        preds_list.extend(preds)
        trues = values.cpu().numpy()
        true_onehot_list.extend(trues)
        # predictions = torch.cat((predictions, outputs), 0) # cat을 통해 예측값을 누적
        # actual = torch.cat((actual, values), 0) # cat을 통해 실제값을 누적

# predictions = predictions.numpy() # 넘파이 배열로 변경
# actual = actual.numpy() # 넘파이 배열로 변경
# rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용하여 RMSE 계산
    
    

In [ ]:
preds_np = np.array(preds_list)
trues_np = np.array(true_onehot_list)


In [ ]:
preds_np.shape

In [ ]:
trues_np.shape

In [ ]:
rmse = np.sqrt(mean_squared_error(preds_np,trues_np))

In [ ]:
mse = np.mean(np.power(preds_np - trues_np, 2), axis=1)

In [ ]:
np.min(mse)


In [ ]:
THRESHOLD = 3

def mad_score(points):
    """https://www.itl.nist.gov/div898/handbook/eda/section3/eda35h.htm """
    m = np.median(points)
    ad = np.abs(points - m)
    mad = np.median(ad)
    
    return 0.6745 * ad / mad

z_scores = mad_score(mse)
# f = lambda x : 0 if (x > THRESHOLD) else 1
# z_scores[]
outliers = np.where(z_scores>THRESHOLD,1,0 )
# z_scores(outliers)
# outliers = f(z_scores)
# outliers = outliers[outliers==1]

In [ ]:
np.unique(outliers)
# print(outliers)

In [ ]:
path = './'
torch.save({'model':model.state_dict(),
            'optimizer':optimizer.state_dict()},
           path+'autoencoder.tar')

In [ ]:
model_load_path = './autoencoder.tar'
checkpoint = torch.load(model_load_path)
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
test_prediction #TODO 앙상블